In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO

import cv2

# Object detection
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Model preparation

In [2]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_08'
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz' 
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/' 

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb' 

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt') 
 
# Class
NUM_CLASSES = 90

# Download Model

In [3]:
fileAlreadyExists = os.path.isfile(PATH_TO_CKPT) 
 
if not fileAlreadyExists: 
    print('Downloading frozen inference graph') 
    opener = urllib.request.URLopener() 
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE) 
    tar_file = tarfile.open(MODEL_FILE) 
    for file in tar_file.getmembers(): 
        file_name = os.path.basename(file.name) 
        if 'frozen_inference_graph.pb' in file_name: 
            tar_file.extract(file, os.getcwd()) 

# Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Loading label map

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Test

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        # Webcam
        cap = cv2.VideoCapture(0)
        while(cap.isOpened()):

            # Capture frame-by-frame
            ret, frame = cap.read()
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(frame, axis=0)
            
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              frame,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)

            # Display the resulting frame
            cv2.imshow('Object Detection',frame)
            if cv2.waitKey(16) & 0xFF == ord('q'):
                break

        # When everything done, release the capture
        print('Colse')
        cap.release()
        cv2.destroyAllWindows() 